In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/JKJ1A
backyard/  data/  model/  notebook/  src/  src_ans/


---
# 概要
ここの内容は[Pytorch公式のチュートリアル](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)に一部基づいています．

Day1-1で，深層学習モデルの学習の全体像を見ました．ここでは，より具体的にそれらを学びます．引き続き，機械学習の三要素を意識してコードを読んでいってください．

**注意（重要）**: コードの全てを完全に理解できない場合は，使い方を把握するように努めてください．「原理はわからないけど，こう使えばこれができる」という理解の仕方も大事です．

---
# 1. データセット準備
画像データセットCIFAR-10をダウンロードし前処理を行います．

CIFAR-10がどのようなデータセットか「CIFAR-10」でGoogle検索してみてください．

In [2]:
# モジュールのインポート
import torch
import torch.nn as nn 
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os 
import warnings
warnings.filterwarnings("ignore")  # warningを表示しない

In [3]:
# データロード関数を定義
## 引数batch_sizeはミニバッチの大きさ．ミニバッチが何かは「ミニバッチ 機械学習」で検索．

# src/cifar10.py で定義されている
def load_data(batch_size=128, n_train=15000, n_test=2500, use_all=False):
    '''
    batch_size: バッチサイズ
    n_train   : 訓練用のデータ数
    n_test    : テスト用のデータ数
    use_all   : 全てのデータを使う場合はTrueを与える
    '''
    # クラスのラベル名
    classes = ('airplane', 'automobile', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    ## 前処理関数の準備
    transform = transforms.Compose([transforms.ToTensor()])

    # CIFAR10の準備（ローカルにデータがない場合はdataディレクトリにダウンロードされる）
    # 訓練用データセット
    trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    # 評価用データセット
    testset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    if not use_all:
        trainset, _ = random_split(trainset, [n_train, len(trainset) - n_train])  # trainsetの内，n_train個だけ選ぶ
        testset, _ = random_split(testset, [n_test, len(testset) - n_test])       # testsetの内，n_test個だけ選ぶ

    # ミニバッチに小分けしておく．これを後で使う
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    # ミニバッチに小分けしておく．これを後で使う
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return (trainloader, testloader, classes)

In [4]:
# 関数を実行
trainloader, testloader, classes = load_data(batch_size=4)

Files already downloaded and verified
Files already downloaded and verified


### Dataloaderの中身
上で，訓練用データと評価用データはそれぞれtrainloaderとtestloaderというDataloaderオブジェクトとして準備された．Dataloaderはイメージとしては，[ミニバッチ1, ミニバッチ2, ...]というようにミニバッチごとに分割されている．

In [ ]:
# trainloaderからミニバッチを一つ取り出す
dataiter = iter(trainloader)
images, labels = next(dataiter)  # ミニバッチを一つ取り出す

'''
今回はバッチサイズ4 (batch_size = 4）なので，
imagesには4枚の画像，labelsには対応する4つのラベルが入っている
'''
print("テンソルのサイズ（形状）．妥当であることを確認せよ")
print(f"images.shape = {images.shape}")  # -> torch.Size([4, 3, 32, 32])
print(f"labels.shape = {labels.shape}")  # -> torch.Size([4])
print('')
'''
画像一枚は，3 x 32 x 32の3次元テンソル
ラベル1つは，サイズ1の1次元テンソル
'''
print("テンソルの中身")
print(images[0]) 
print(labels[0]) 

### 画像の表示

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# !!画像を表示するための関数
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 画像の表示
imshow(torchvision.utils.make_grid(images))
# ラベルのプリント
print(' '.join('%5s' % classes[labels[j]] for j in range(images.shape[0])))

---
# 2. モデルの定義
学習させるニューラルネットワークのモデルを定義します．

In [32]:
import torch.nn as nn
import torch.nn.functional as F

## ネットワークのクラス
class LeNet(nn.Module):                          # 「ルネット」という名前のネットワーク構造
    # 初期化の時にレイヤーが準備される部分
    def __init__(self):   
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)          # 畳み込み層
        self.pool = nn.MaxPool2d(2, 2)           # プーリング層
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)    # 全結合層   
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # ネットワークにデータを通す時に機能する
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))     # 畳み込み -> 活性化 -> プーリング
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)               # テンソルの形を成形
        x = F.relu(self.fc1(x))                  # 全結合 -> 活性化
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

**実際に具体的なモデルを生成**

In [31]:
model = LeNet()   # __init__(self)の部分が実行される

**モデルにデータを代入する**

In [ ]:
it = iter(trainloader)
images, labels = next(it)  # ミニバッチを一つ取り出す

output = model(images)  # forward(self, x)の部分が実行される（model(x)の代わりにmodel.forward(x)でもOK）
                      # ミニバッチの画像4枚をまとめて代入している
                      # ネットワークの出力は10次元ベクタ（を画像4枚分）

'''（重要）
具体的には以下のような 画像の枚数（4）x ネットワークの出力次元（10）
output = [[-0.03, 0.08, ..., 0.06], 
          [-0.04, 0.06, ..., 0.03], 
          [-0.03, 0.10, ..., 0.02], 
          [-0.01, 0.09, ..., 0.04]]
 - 画像の枚数が4なのはbatch_size = 4と設定したから．
 - ネットワークの出力次元が10なのは，今回は10クラスの分類問題だから．
'''
print(f'output.shape = {output.shape}')  # -> torch.Size([4, 10])

**分類結果を見てみる**

In [10]:
'''（重要）
          ----> dim=1
output = :     [[-0.03, 0.08, ..., 0.06], 
         v    　[-0.04, 0.06, ..., 0.03], 
        dim0    [-0.03, 0.10, ..., 0.02], 
                [-0.01, 0.09, ..., 0.04]]
 - 各画像のネットワークの出力（10次元）のうち，「最大の値がある位置」が，ネットワークによる予測のラベル．
 - 「最大の値」は `max()` という関数で見つけられる．「最大の値がある位置」は `argmax()`という関数で見つけられる．
 - 特に今回は横方向（dim=1の方向）で最大を見たいので，`argmax(dim=-1)`という関数を使う．
'''
pred_labels = output.argmax(dim=1)  # これは4次元ベクトル

'''
またネットワークを学習させていないので，正解のラベルと全く合っていないことが見て取れる
'''
print(f'predicted labels: {pred_labels}')
print(f'true labels     : {labels}')

predicted labels: tensor([7, 7, 7, 7])
true labels     : tensor([7, 4, 3, 0])


---
# 3. 学習
準備したデータセットでモデルを学習します．Trainerクラスを作ると良いでしょう．

### 訓練

In [29]:
class Trainer():
    def __init__(self, params=None):
        '''
        params: 訓練に関する設定をもつ辞書
        '''
        self.params = params
        self.model = self.get_model(params['model_name'])
        self.optimizer = self.get_optimizer(self.model.parameters(), params['learning_rate'])
        self.criterion = self.get_criterion()            
        
    def get_model(self, model_name='LeNet'):
        if model_name == 'LeNet': 
            model = LeNet() 
        if model_name == 'VGG11':
            model = VGG11()
        return model.cuda()

    def get_optimizer(self, parameters, learning_rate):
        optimizer = torch.optim.SGD(parameters, lr=learning_rate)  # オプティマイザ定義（確率的勾配法）
        return optimizer
    
    def get_criterion(self): 
        criterion = nn.CrossEntropyLoss()  # 分類で標準的なクロスエントロピーロス  
        return criterion
        
    def train(self, trainloader, testloader=None):
        ''' 
        練習課題1
        Day-1-1-overview.ipynbを参考に埋めよ．train_step()も必要になる．
        '''
        
    def evaluate(self, testloader):
        '''
        練習課題1
        受け取ったtestloaderにあるデータをモデルに与え，その分類結果の正答率を返したい．
        一つ前の「分類結果を見てみる」のセルを参考に埋めよ．
        可能なら，この関数をtrainの中にも入れて，エポックごとにテストデータの正答率も表示されるようにせよ．
        後方で登場する，「4. モデルの評価」も参考になる
        '''
        n_hits, n_samples = 0, 0
        
        # この間を埋めよ
        
        accuracy = n_hits / n_samples
        return accuracy
    
    def save_model(self, save_path):
        '''
        完成させよ（後述の練習課題2参照）
        '''


### 練習課題１
1. Trainerクラスのtrainメソッドを完成させよ
2. Trainerクラスのevaluateメソッドを完成させよ

In [ ]:
params = {'model_name': 'LeNet', 
          'n_epochs': 10,
          'learning_rate': 0.01
          }
trainer = Trainer(params)
trainer.train()               # 訓練の実行
model = trainer.model         # モデルを取り出す
model = model.eval()          # 学習をしない場合は`.eval()`で「評価モード」に切り替える．

### 学習したモデルの保存

In [ ]:
PATH = 'results/model_cifar10.pth'
os.makedirs('results/', exist_ok=True) # modelディレクトリの作成（一回だけ行ったらコメントアウト）

torch.save(model.state_dict(), PATH)
# %ls results/    # resultsディレクトリの中を表示

'''（重要）読み込むときは以下'''
model = LeNet()                          # 空のネットワークを準備
state_dict = torch.load(PATH)          # 保存したパラメタをロード
model.load_state_dict(state_dict)      # ネットワークにパラメタをセット
model = model.eval()                   # 学習をしない場合は`.eval()`で「評価モード」に切り替える．

### 練習課題２
上を参考に，Trainerクラスに，save_modelメソッドを追加せよ．
- `trainer.train()`で訓練をした後に，`trainer.save_model(PATH)`としたい（`PATH`は保存先）．

---
# 4. モデルの評価
学習したモデルの性能を評価します．

### 練習問題: 学習したモデルを評価してみよう
学習したモデルの正解率を計算する関数`evaluate()`を定義して，訓練データ，テストデータそれぞれでの正解率を出してみましょう．

In [ ]:
%%time

'''
このセルが正しく実行され，分類精度が表示されたら成功
'''

# テストデータでの正解率
test_acc = evaluate(model, testloader)   
print(f' test acc = {test_acc}')

### 練習問題（解答例）

In [23]:
def evaluate(model, dataloader):
    model.eval()  # ネットワークを「評価モード」にする

    correct = 0  # 正解数
    total = 0    # 画像総数

    for data in dataloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()

        outputs = model(images)
        predicted = outputs.argmax(dim=1)  # !各画像に関して最大値のインデックスを取り出す
        
        n_hits += (predicted == labels).sum().item()  # predicts == labelsは各要素を比較して，[True, False, True, ...]みたいなのを返す．
        total += len(labels)

    acc = n_hits / total

    return acc 

In [ ]:
%%time 
train_acc = evalute(model, trainloader)
test_acc  = evalaute(model, testloader)
print(f'train acc = {train_acc:.3f}')  # ':.3f'とつけると小数点以下3桁までの表示になる
print(f' test acc = {test_acc:.3f}')

## 確認 モデルの学習，評価，保存
以上の練習課題を終えると，以下のコードで学習の学習，評価，保存が行えるはず

In [ ]:
import yaml 
params = {'model_name': 'LeNet', 
          'n_epochs': 10,
          'learning_rate': 0.01,
          'batch_size': 256,
          'dataset': 'cifar10',
          'env_name': 'dryrun'
          }
save_dir = os.path.join('results', params['env_name']) # 保存先は，results/dryrun/となる
os.makedirs(save_dir, exist_ok=True)  # フォルダの作成

trainloader, testloader, classes = load_data(batch_size=params['batch_size'], use_all=False)  # 適宜use_all=Trueにする

trainer = Trainer(params)

# モデルの学習・評価
trainer.train(trainloader, testloader)               # 訓練の実行
trainer.evaluate(testloader)

# モデルの保存
model_file = os.path.join(save_dir, f"{params['model_name']}_{params['dataset']}.pth")
trainer.save_model(model_file)  

# 学習設定の保存
setting_file = os.path.join(save_dir, f"{params['model_name']}_{params['dataset']}.yml")
with open(setting_file, 'w') as yaml_file:
    yaml.dump(params, yaml_file)

---
# 5. `.py`ファイルで学習

今使っているセルごとに実行している形式はJupyter notebookと呼ばれます．これは便利ですが，コードが長くなると，たくさんのセルに散らばるし，いちいち全てを実行するのも面倒になります．全ての内容をひとまとめにしたPythonファイル `train_cifar10.py`を作ると，次のように実行できてとても便利です．


In [ ]:
!python src/train_cifar10.py --nepochs 2 --batch_size 256 --lr 0.01 --env_name 'dryrun'

上に書いてあるのを説明します．
- `!python src/train_cifar10.py`で，`src`ディレクトリにある`train_cifar10.py`を実行します．
- `--nepochs 2`などで訓練パラメータを指定しています．
- `--env_name 'dryrun'`によって，`results/dryrun`以下にモデルや設定ファイルが保存されます

### Day1課題
上のセルが実行できるよう，train_cifar10.pyを完成させよ